# Split Subjective Days

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import seaborn as sns
import pylab as py
import statistics

# Summary Filename

In [3]:
# Read Spider FRP CSV file
summary_filename = 'Metazygia wittfeldae summary'
summary_df = pd.read_csv(summary_filename + '.csv')

# Get FRP Value of Specific Spider

In [15]:
# Extract FRP values for each spider and create new CSV file

def get_frp(summary_df, spider_name):
    # Creating new FRP CSV file from summary CSV
    summary_df = summary_df[(summary_df['Conditions'] == 'DD') & (summary_df['Spider ID'].str.match('Monitor 1'))]
    frp_df = summary_df[['Spider ID', 'LombSc period']]
    frp_df = frp_df.reset_index(drop = True)
    frp_df = frp_df.set_index('Spider ID')
    frp_df.to_csv(summary_filename + '_period')
    
    # Get FRP value of specific spider
    frp = float(frp_df.values[frp_df.index.str.endswith(spider_name)])
    
    return frp

get_frp(summary_df, 'Monitor 1 Spider 1')

22.0111503

# Activity Filename

In [5]:
# Read spider activity CSV file

filename = 'Metazygia wittfeldae Monitor 1 Updated_DD_binary'
df = pd.read_csv(filename + '.csv', index_col = 0)
df.index = pd.to_datetime(df.index)
#display(df)

# Split Subjective Days Function

Given arguments dataframe in DateTimeIndex, the initial day where the lights didn't turn on, and the FRP of the spider, returns the subjective days of the spider.

In [38]:
def split_subjective_days(df, initial_day, frp):
    # Get day where lights would turn on but they don't (DD)
    df_start = df.at_time('07:00:00')
    idx = df_start.index[0]
    
    # Subjective days starting from lights not being turned on to given FRP
    dti = pd.date_range(idx, periods = 10, freq = frp)
    dti = dti.round(freq = '1T')
    
    return dti

split_subjective_days(df, idx, str(frp) + 'H')

NameError: name 'frp' is not defined

# Extracting Activity

In [18]:
def extract_activity_from_subjective_day(dti, start_index, end_index, column_name):
    # Extract activity based on subjective day index
    activitydf = df.loc[dti[start_index] : dti[end_index], [column_name]]
    
    return activitydf

extract_activity_from_subjective_day(dti, 0, 1, 'Monitor 1 Spider 1')

,Monitor 1 Spider 1
Date-Time,
2017-04-26 07:00:00,0
2017-04-26 07:01:00,0
2017-04-26 07:02:00,0
2017-04-26 07:03:00,0
2017-04-26 07:04:00,0
...,...
2017-04-27 04:57:00,0
2017-04-27 04:58:00,0
2017-04-27 04:59:00,0


# Get Probability

In [26]:
def probability_in_sections(activitydf, start_index, end_index, column_name):
    list_of_dates = extract_activity_from_subjective_day(dti, start_index, end_index, column_name).index.tolist()
    median_date = statistics.median_low(list_of_dates)
    #activitydf = extract_activity_from_subjective_day(dti, start_index, end_index, column_name)
    
    sub_day = activitydf.loc[dti[start_index] : median_date]
    rows = len(sub_day.index)
    total_day_movement = sub_day.sum()
    prob_day = total_day_movement / rows
    
    sub_night = activitydf.loc[median_date : dti[end_index]]
    rows = len(sub_night.index)
    total_night_movement = sub_night.sum()
    prob_night = total_night_movement / rows
    
    return prob_day, prob_night

In [28]:
probability_in_sections(extract_activity_from_subjective_day(dti, 0, 1, 'Monitor 1 Spider 1'), 0, 1, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.028744
 dtype: float64,
 Monitor 1 Spider 1    0.270393
 dtype: float64)

In [29]:
probability_in_sections(extract_activity_from_subjective_day(dti, 1, 2, 'Monitor 1 Spider 1'), 1, 2, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.027231
 dtype: float64,
 Monitor 1 Spider 1    0.23298
 dtype: float64)

In [30]:
probability_in_sections(extract_activity_from_subjective_day(dti, 2, 3, 'Monitor 1 Spider 1'), 2, 3, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.009077
 dtype: float64,
 Monitor 1 Spider 1    0.261329
 dtype: float64)

In [31]:
probability_in_sections(extract_activity_from_subjective_day(dti, 3, 4, 'Monitor 1 Spider 1'), 3, 4, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.0
 dtype: float64,
 Monitor 1 Spider 1    0.187311
 dtype: float64)

In [32]:
probability_in_sections(extract_activity_from_subjective_day(dti, 4, 5, 'Monitor 1 Spider 1'), 4, 5, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.001513
 dtype: float64,
 Monitor 1 Spider 1    0.217852
 dtype: float64)

In [33]:
probability_in_sections(extract_activity_from_subjective_day(dti, 5, 6, 'Monitor 1 Spider 1'), 5, 6, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.04236
 dtype: float64,
 Monitor 1 Spider 1    0.132931
 dtype: float64)

In [34]:
probability_in_sections(extract_activity_from_subjective_day(dti, 6, 7, 'Monitor 1 Spider 1'), 6, 7, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.065053
 dtype: float64,
 Monitor 1 Spider 1    0.13142
 dtype: float64)

In [35]:
probability_in_sections(extract_activity_from_subjective_day(dti, 7, 8, 'Monitor 1 Spider 1'), 7, 8, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.062027
 dtype: float64,
 Monitor 1 Spider 1    0.098336
 dtype: float64)

In [36]:
probability_in_sections(extract_activity_from_subjective_day(dti, 8, 9, 'Monitor 1 Spider 1'), 8, 9, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.068079
 dtype: float64,
 Monitor 1 Spider 1    0.064955
 dtype: float64)

# Trying to Loop

In [25]:
start_index = 0
end_index = start_index + 1

for start_index, end_index in enumerate(dti):
    print(probability_in_sections(extract_activity_from_subjective_day(dti, start_index, end_index, 
                            'Monitor 1 Spider 1'), start_index, end_index, 'Monitor 1 Spider 1'))
    start_index += 1

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

# Testing

In [13]:
df_start = df.at_time('07:00:00')
idx = df_start.index[0]
    
# Subjective days starting from lights not being turned on to given FRP
dti = pd.date_range(idx, periods = 10, freq = '22.0111503H')
dti = dti.round(freq = '1T')
#print(dti)

start_index = 0
end_index = 1

list_of_dates = extract_activity_from_subjective_day(dti, start_index, end_index, 'Monitor 1 Spider 1').index.tolist()
median_date = statistics.median_low(list_of_dates)

activitydf = df.loc[dti[start_index] : dti[end_index], ['Monitor 1 Spider 1']]
sub_day = activitydf.loc[dti[start_index] : median_date]
#display(sub_day)
rows = len(sub_day.index)
total_day_movement = sub_day.sum()
prob_day = total_day_movement / rows
print(prob_day)
    
sub_night = activitydf.loc[median_date : dti[end_index]]
rows = len(sub_night.index)
total_night_movement = sub_night.sum()
prob_night = total_night_movement / rows
print(prob_night)

start_index = 0
end_index = 1

for start_index, end_index in enumerate(dti):
    list_of_dates = extract_activity_from_subjective_day(dti, start_index, end_index, 'Monitor 1 Spider 1').index.tolist()
    median_date = statistics.median_low(list_of_dates)

    activitydf = df.loc[dti[start_index] : dti[end_index], ['Monitor 1 Spider 1']]
    sub_day = activitydf.loc[dti[start_index] : median_date]
    display(sub_day)
    rows = len(sub_day.index)
    total_day_movement = sub_day.sum()
    prob_day = total_day_movement / rows
    print(prob_day)
    
    sub_night = activitydf.loc[median_date : dti[end_index]]
    rows = len(sub_night.index)
    total_night_movement = sub_night.sum()
    prob_night = total_night_movement / rows
    print(prob_night)

NameError: name 'extract_activity_from_subjective_day' is not defined